In [56]:
import tkinter as tk
from tkinter import messagebox, ttk
import mysql.connector
from datetime import datetime
def conn():
    return mysql.connector.connect(
        host = 'localhost',
        user = 'root',
        password = 'avinashA@1',
        database = 'expense_tracker'
    )

# def add_expense():
#     category = input("Enter Category of expense like,grocery,medicines,bills etc...")
#     discription = input("Enter description [optional]")
#     amount = float(input("Enter Amount Spent"))
    # date_str = input("If not todays date enter date in format yyyy-mm-dd")
    # date = date_str if date_str else datetime.today().strftime('%Y-%m-%d')
    # strftime - funtion of datetime module which converts date to string format
    # shorthand if - if date_str is empty will return false
    # con = conn()
    # cursor = con.cursor()
    # cursor.execute('''insert into expense(date,amount,category,description)
    # values (%s,%s,%s,%s)''',(date,amount,category,discription))
    # con.commit()
    # print("added Successfully")
 
    
# def display():
#     con = conn()
#     cursor = con.cursor()
#     cursor.execute("select * from expense")
#     myrecords = cursor.fetchall()
#     for res in myrecords:
#         print(res)
#     cursor.close()
#     con.close()

# def main():
#     while True:
#         print("\n---Select one of the given---")
#         print("1. Add An Expense")
#         print("2. Delete Expense")
#         print("3. View All Expenses")
#         print("4. Get Monthly Summary")
#         print("5. View Records")
#         print("6. Exit")
#         print("\n")
#         choice = input("Enter Your Choice")
#         if choice =='1':
#             add_expense()
#         elif choice =='5':
#             print("********* All Records ***********")
#             display()
#         elif choice == '6':
#             print("Thank you for using")
#             break;
#         else:
#             print("Not a Correct Choice!! Please Re-enter")
# ---------- Add Expense to DB ----------
def add_expense_to_db(date, amount, category, description,location):
    try:
        con = conn()
        cursor = con.cursor()
        cursor.execute(
            "INSERT INTO expense (date, amount, category, description,location) VALUES (%s, %s, %s, %s,%s)",
            (date, amount, category, description,location)
        )
        con.commit()
        cursor.close()
        con.close()
        return True
    except mysql.connector.Error as err:
        messagebox.showerror("Database Error", f"Error: {err}")
        return False
# ---------- Load Expenses ----------
def load_expenses():
    for row in tree.get_children():
        tree.delete(row)

    con = conn()
    cursor = con.cursor()
    cursor.execute("SELECT id, date, amount, category, description ,location FROM expense ORDER BY date DESC")
    records = cursor.fetchall()
    for index, record in enumerate(records):
        tag = 'evenrow' if index % 2 == 0 else 'oddrow'
        tree.insert("", "end", values=record, tags=(tag,))
    cursor.close()
    con.close()


# ---------- Add Expense Button Action ----------
def add_expense():
    category = category_var.get()
    description = description_entry.get()
    amount = amount_entry.get()
    date = date_entry.get()
    location =location_dropdown.get()
    if not category or not amount:
        messagebox.showwarning("Input Error", "Category and Amount are required.")
        return

    try:
        amount = float(amount)
    except ValueError:
        messagebox.showerror("Input Error", "Amount must be a number.")
        return

    if not date:
        date = datetime.today().strftime('%Y-%m-%d')
    else:
        try:
            datetime.strptime(date, '%Y-%m-%d')
        except ValueError:
            messagebox.showerror("Input Error", "Date must be in YYYY-MM-DD format.")
            return

    if add_expense_to_db(date, amount, category, description,location):
        messagebox.showinfo("Success", "Expense added successfully!")
        clear_inputs()
        load_expenses()
# ---------- Delete Selected Expense ----------
def delete_selected():
    selected_item = tree.selection()
    if not selected_item:
        messagebox.showwarning("No Selection", "Please select a record to delete.")
        return

    record = tree.item(selected_item[0])["values"]
    expense_id = record[0]  # ID is the first column

    confirm = messagebox.askyesno("Confirm Deletion", f"Delete record ID {expense_id}?")
    if not confirm:
        return

    try:
        con = conn()
        cursor = con.cursor()
        cursor.execute("DELETE FROM expense WHERE id = %s", (expense_id,))
        con.commit()
        cursor.close()
        con.close()
        messagebox.showinfo("Deleted", f"Record ID {expense_id} deleted.")
        load_expenses()
    except mysql.connector.Error as err:
        messagebox.showerror("Database Error", f"Error: {err}")

def clear_inputs():
    category_var.set("")
    description_entry.delete(0, tk.END)
    amount_entry.delete(0, tk.END)
    date_entry.delete(0, tk.END)

# ---------- GUI Setup ----------
root = tk.Tk()
root.title(" 💸 Expense Tracker")
root.geometry("1250x700")
root.configure(bg="#F7F7F7")
root.resizable(False, False)

# ---------- Style ----------
style = ttk.Style()
style.theme_use("clam")  # Alternative: 'alt', 'default', 'classic'
style.configure("Treeview", 
    font=("Segoe UI", 10),
    rowheight=25,
    background="#FAFAFA",
    fieldbackground="#FAFAFA"
)
style.configure("Treeview.Heading", 
    font=("Segoe UI", 11, "bold"),
    background="#333333",
    foreground="white"
)
style.map("Treeview", background=[('selected', '#2E86C1')])

# Row coloring
tree_tag_styles = {
    'evenrow': {'background': '#E8F6F3'},
    'oddrow': {'background': '#D6EAF8'}
}
# Style for Combobox
style.configure("TCombobox",
                fieldbackground="white",
                background="white",
                foreground="#2c3e50",
                font=("Segoe UI", 11),
                padding=5)

# Style for Label
label_style = {
    "font": ("Segoe UI", 11, "bold"),
    "bg": "#f0f4f8",
    "fg": "#34495e"
}
style = ttk.Style()
style.theme_use('clam')  # 'clam' allows more styling options
label_style = {
    "font": ("Segoe UI", 11, "bold"),
    "bg": "#f0f4f8",
    "fg": "#34495e"
}
# ---------- Header ----------
header = tk.Label(root, text="💸 Expense Tracker", font=("Segoe UI", 20, "bold"), bg="#F7F7F7", fg="#2C3E50")
header.pack(pady=10)


tk.Label(root, text="Select Location:", **label_style).pack(pady=(25, 5))
selected_location = location_var.get()
location = ["Hospital", "Home", "Office", "Workshop"]
location_var = tk.StringVar()
location_dropdown = ttk.Combobox(root, textvariable=location_var, values=location, state="readonly", width=30)
location_dropdown.pack(pady=10)
location_dropdown.current(1) # Default to empty

# ---------- Input Frame ----------
input_frame = tk.Frame(root, bg="#F7F7F7")
input_frame.pack(pady=10, padx=50, fill="x")

tk.Label(input_frame, text="Category:", font=("Segoe UI", 11), bg="#F7F7F7").grid(row=0, column=0, sticky="e")
category_var = tk.StringVar()
category_entry = tk.Entry(input_frame, textvariable=category_var, font=("Segoe UI", 10), width=20)
category_entry.grid(row=0, column=1, padx=5)

tk.Label(input_frame, text="Description:", font=("Segoe UI", 11), bg="#F7F7F7").grid(row=0, column=2, sticky="e")
description_entry = tk.Entry(input_frame, font=("Segoe UI", 10), width=30)
description_entry.grid(row=0, column=3, padx=5)

tk.Label(input_frame, text="Amount:", font=("Segoe UI", 11), bg="#F7F7F7").grid(row=1, column=0, sticky="e")
amount_entry = tk.Entry(input_frame, font=("Segoe UI", 10), width=20)
amount_entry.grid(row=1, column=1, padx=5)

tk.Label(input_frame, text="Date (YYYY-MM-DD):", font=("Segoe UI", 11), bg="#F7F7F7").grid(row=1, column=2, sticky="e")
date_entry = tk.Entry(input_frame, font=("Segoe UI", 10), width=30)
date_entry.grid(row=1, column=3, padx=5)

# ---------- Add Button ----------
add_btn = tk.Button(input_frame, text="➕ Add Expense", font=("Segoe UI", 10, "bold"), bg="#27AE60", fg="white", command=add_expense)
add_btn.grid(row=2, column=3, pady=15, sticky="e")

# ---------- Treeview Frame ----------
tree_frame = tk.Frame(root)
tree_frame.pack(padx=20, fill="both", expand=True)

columns = ("ID", "Date", "Amount", "Category", "Description","Location")
tree = ttk.Treeview(tree_frame, columns=columns, show="headings", height=10)

# Define column properties
for col in columns:
    tree.heading(col, text=col)
    tree.column(col, anchor="center", stretch=True)

# Scrollbar
scrollbar = ttk.Scrollbar(tree_frame, orient="vertical", command=tree.yview)
tree.configure(yscroll=scrollbar.set)
scrollbar.pack(side="right", fill="y")
tree.pack(fill="both", expand=True)

# Add tag styles
for tag, style_config in tree_tag_styles.items():
    tree.tag_configure(tag, **style_config)

delete_btn = tk.Button(root, text="🗑 Delete Selected", font=("Segoe UI", 10, "bold"), bg="#E74C3C", fg="white", command=delete_selected)
delete_btn.pack(pady=10)
# ---------- Load Data ----------
load_expenses()

# ---------- Mainloop ----------
root.mainloop()